# Procedimiento para verificar correctitud en generación de archivo yyyy-mm-dd.shape

In [71]:
import csv
import os
from collections import defaultdict

import decouple
import pandas as pd
import plotly.express as px
import utm

from op_processing_for_vis.config import OUTPUT_PATH, TMP_PATH
from op_processing_for_vis.utils import get_route_id_info

config = decouple.AutoConfig(' ')

In [72]:
# load processed shape
op_date = '2022-07-02'
filename = '{}.shape'.format(op_date)
shape_path = os.path.join(OUTPUT_PATH, op_date, filename)

shape_dict = defaultdict(list)
with open(shape_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter='|')
    for row in reader:
        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        is_starting_segment = int(row['IsSectionInit'])
        shape_dict[row['Route']].append((longitude, latitude, is_starting_segment))

In [ ]:
# load original shape
route_id_info = get_route_id_info(op_date)
original_shape_dict = defaultdict(list)
original_shape_path = os.path.join(TMP_PATH, '00Entrada', op_date, 'Rutas',
                                   'ShapeRutas_{0}.csv'.format(op_date.replace('-', '')))
with open(original_shape_path, newline='', encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        route_id = row['ROUTE_NAME']
        x = float(row['X-Coordinate'])
        y = float(row['Y-Coordinate'])
        latitude, longitude = utm.to_latlon(x, y, 19, 'H')
        auth_route_code = route_id_info[route_id]['auth_route_code']
        original_shape_dict[auth_route_code].append((longitude, latitude))

In [ ]:
def generate_shape_evaluation_map(shape_points, filename, original_shape=None, auto_open=False):
    mapbox_access_token = config('MAPBOX_ACCESS_TOKEN')

    # shape
    df = pd.DataFrame(shape_points, columns=['lon', 'lat', 'is_starting_segment'])
    fig = px.line_mapbox(df, lat='lat', lon='lon', color_discrete_sequence=['blue'],
                         zoom=11, height=700, title="")

    # original shape
    if original_shape is not None:
        df5 = pd.DataFrame(original_shape, columns=['lon', 'lat'])
        fig5 = px.line_mapbox(df5, lat='lat', lon='lon', color_discrete_sequence=['green'])

    # all points
    df2 = pd.DataFrame(shape_points, columns=['lon', 'lat', 'is_starting_segment'])
    fig2 = px.scatter_mapbox(df2, lat='lat', lon='lon', hover_name='is_starting_segment',
                             color_discrete_sequence=['yellow'], hover_data=['lon', 'lat', 'is_starting_segment'])

    # end shape point
    df3 = pd.DataFrame.from_records([shape_points[-1]], columns=['lon', 'lat', 'is_starting_segment'])
    fig3 = px.scatter_mapbox(df3, lat='lat', lon='lon', color_discrete_sequence=['blue'], size=[40])

    # starting segment points
    df4 = pd.DataFrame(shape_points, columns=['lon', 'lat', 'is_starting_segment'])
    df4 = df4[df['is_starting_segment'] == 1]
    fig4 = px.scatter_mapbox(df4, lat='lat', lon='lon', hover_name='is_starting_segment',
                             color_discrete_sequence=['red'], hover_data=['lon', 'lat', 'is_starting_segment'])

    if original_shape is not None:
        fig.add_trace(fig5.data[0])
    fig.add_trace(fig2.data[0])
    fig.add_trace(fig3.data[0])
    fig.add_trace(fig4.data[0])

    fig.update_layout(mapbox_style="dark", mapbox_accesstoken=mapbox_access_token)
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    #fig.show()
    fig.write_html(os.path.join(TMP_PATH, 'maps', '{0}.html'.format(filename)), auto_open=auto_open)

In [ ]:
# example
auth_route_code = 'B01 00I'
points = shape_dict[auth_route_code]
original_points = original_shape_dict[auth_route_code]

generate_shape_evaluation_map(points, auth_route_code, original_shape=None)